<a href="https://colab.research.google.com/github/bharathbolla/EVA6/blob/S-6/BN_vs_GN_vs_LN_Assisgnment_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import seaborn as sns
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/Colab Notebooks

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks


In [2]:
# this performs the import from the model.py file
import model



In [3]:
from model import Model, BatchNet, GroupNet, LayerNet

In [4]:

nn_model = Model(BatchNet(), GroupNet(), LayerNet())

In [5]:
params = list(nn_model.GN.parameters())
for p in params:
  print(p.size())

torch.Size([8, 1, 3, 3])
torch.Size([8])
torch.Size([8])
torch.Size([12, 8, 3, 3])
torch.Size([12])
torch.Size([12])
torch.Size([8, 12, 1, 1])
torch.Size([12, 8, 3, 3])
torch.Size([12])
torch.Size([12])
torch.Size([16, 12, 3, 3])
torch.Size([16])
torch.Size([16])
torch.Size([12, 16, 1, 1])
torch.Size([15, 12, 3, 3])
torch.Size([15])
torch.Size([15])
torch.Size([15, 15, 3, 3])
torch.Size([15])
torch.Size([15])
torch.Size([10, 15, 1, 1])


In [6]:
params = list(nn_model.BN.parameters())
for p in params:
  print(p.size())

torch.Size([8, 1, 3, 3])
torch.Size([8])
torch.Size([8])
torch.Size([12, 8, 3, 3])
torch.Size([12])
torch.Size([12])
torch.Size([8, 12, 1, 1])
torch.Size([12, 8, 3, 3])
torch.Size([12])
torch.Size([12])
torch.Size([16, 12, 3, 3])
torch.Size([16])
torch.Size([16])
torch.Size([12, 16, 1, 1])
torch.Size([15, 12, 3, 3])
torch.Size([15])
torch.Size([15])
torch.Size([15, 15, 3, 3])
torch.Size([15])
torch.Size([15])
torch.Size([10, 15, 1, 1])


In [7]:
params = list(nn_model.BN.parameters())
for p in params:
  print(p.size())

torch.Size([8, 1, 3, 3])
torch.Size([8])
torch.Size([8])
torch.Size([12, 8, 3, 3])
torch.Size([12])
torch.Size([12])
torch.Size([8, 12, 1, 1])
torch.Size([12, 8, 3, 3])
torch.Size([12])
torch.Size([12])
torch.Size([16, 12, 3, 3])
torch.Size([16])
torch.Size([16])
torch.Size([12, 16, 1, 1])
torch.Size([15, 12, 3, 3])
torch.Size([15])
torch.Size([15])
torch.Size([15, 15, 3, 3])
torch.Size([15])
torch.Size([15])
torch.Size([10, 15, 1, 1])


/content/model.py

In [8]:


for param in nn_model.parameters():
    print( param.size())

torch.Size([8, 1, 3, 3])
torch.Size([8])
torch.Size([8])
torch.Size([12, 8, 3, 3])
torch.Size([12])
torch.Size([12])
torch.Size([8, 12, 1, 1])
torch.Size([12, 8, 3, 3])
torch.Size([12])
torch.Size([12])
torch.Size([16, 12, 3, 3])
torch.Size([16])
torch.Size([16])
torch.Size([12, 16, 1, 1])
torch.Size([15, 12, 3, 3])
torch.Size([15])
torch.Size([15])
torch.Size([15, 15, 3, 3])
torch.Size([15])
torch.Size([15])
torch.Size([10, 15, 1, 1])
torch.Size([8, 1, 3, 3])
torch.Size([8])
torch.Size([8])
torch.Size([12, 8, 3, 3])
torch.Size([12])
torch.Size([12])
torch.Size([8, 12, 1, 1])
torch.Size([12, 8, 3, 3])
torch.Size([12])
torch.Size([12])
torch.Size([16, 12, 3, 3])
torch.Size([16])
torch.Size([16])
torch.Size([12, 16, 1, 1])
torch.Size([15, 12, 3, 3])
torch.Size([15])
torch.Size([15])
torch.Size([15, 15, 3, 3])
torch.Size([15])
torch.Size([15])
torch.Size([10, 15, 1, 1])
torch.Size([8, 1, 3, 3])
torch.Size([8])
torch.Size([8])
torch.Size([12, 8, 3, 3])
torch.Size([12])
torch.Size([12])
tor

## Data Transformations

We first start with defining our data transformations. We need to think what our data is and how can we augment it to correct represent images which it might not see otherwise. 


In [9]:
train_transforms = transforms.Compose([
                                   transforms.RandomRotation((-12.0, 12.0), fill=(1,)),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.1307,), (0.3081,)) # The mean and std have to be sequences (e.g., tuples), therefore you should add a comma after the values. 
                                   ])

test_transforms = transforms.Compose([
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.1307,), (0.3081,))
                                   ])

# Dataset and Creating Train/Test Split

In [10]:
train = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

# Dataloader Arguments & Test/Train Dataloaders


In [11]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

CUDA Available? True


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# The model
Let's start with the model we first saw

##### Train and Test functions

In [12]:
# creating the train and
from tqdm import tqdm

train_losses_BN = []
train_losses_GN = []
train_losses_LN = []

test_losses_BN = []
test_losses_GN = []
test_losses_LN = []

train_acc_BN = []
train_acc_GN = []
train_acc_LN = []

test_acc_BN = []
test_acc_GN = []
test_acc_LN = []


train_losses_epoch_BN = []
train_losses_epoch_GN = []
train_losses_epoch_LN = []

test_losses_epoch_BN = []
test_losses_epoch_GN = []
test_losses_epoch_LN = []

train_acc_epoch_BN = []
train_acc_epoch_GN = []
train_acc_epoch_LN = []

test_acc_epoch_BN = []
test_acc_epoch_GN = []
test_acc_epoch_LN = []


def train(model, device, train_loader, optimizer_BN, optimizer, GN, optimizer_LN, epoch, lamnda_l1 = 0.01, l1= False):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  incorrect = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    data, target = data.to(device), target.to(device)
    
    y_pred_BN, y_pred_GN, y_pred_LN = model(data)
    
    ##BN

    optimizer_BN.zero_grad()  
    loss_BN = F.nll_loss(y_pred_BN, target)
    loss_BN.backward()
    optimizer_BN.step()
    train_losses_BN.append(loss)
    
    # GN
    optimizer_GN.zero_grad()
    loss_GN = F.nll_loss(y_pred_GN, target)
    loss_GN.backward()
    optimizer_GN.step()
    train_losses_GN.append(loss)

    #3 LN   
    # Calculate loss----------------------------------------------------------------------------------
    optimizer_LN.zero_grad()
    loss_LN = F.nll_loss(y_pred_LN, target)
    loss_LN.backward()
    optimizer_LN.step() 
    train_losses_LN.append(loss)   

    
    
    
    # l1 = 0
    # if l1 == True:
    #     for p in Model.GN.parameters:
    #         l1 = l1 + p.abs().sum()
    # # adding the l1 loss
    #     loss_ = loss + lambda_l1 *l1   
    # --------------------------------------------------------------------------------------------------------------------------------------------------------------------
    # Backpropagation


    # Update pbar-tqdm    
    pred_BN = y_pred_BN.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    pred_GN = y_pred_GN.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    pred_LN = y_pred_LN.argmax(dim=1, keepdim=True)  # get the index of the max log-probability

    correct_BN += pred_BN.eq(target.view_as(pred_BN)).sum().item()
    correct_GN += pred_GN.eq(target.view_as(pred_GN)).sum().item()
    correct_LN += pred_LN.eq(target.view_as(pred_LN)).sum().item()
    
    processed+= len(data)


    # finding out incorrect values
    # batch_incorrect = processed - correct
    # incorrect  = incorrect + batch_incorrect

    pbar.set_description(desc= f'Loss={loss_BN.item()} Batch_id={batch_idx} Accuracy={100*correct_BN/processed:0.2f}')
    pbar.set_description(desc= f'Loss={loss_GN.item()} Batch_id={batch_idx} Accuracy={100*correct_GN/processed:0.2f}')
    pbar.set_description(desc= f'Loss={loss_LN.item()} Batch_id={batch_idx} Accuracy={100*correct_LN/processed:0.2f}')

    train_acc_BN.append(100*correct_BN/processed)
    train_acc_GN.append(100*correct_GN/processed)
    train_acc_LN.append(100*correct_LN/processed)


    # # finding out incorrect values
    # batch_incorrect = processed - correct
    # incorrect  = incorrect + batch_incorrect

  train_losses_epoch_BN.append(sum(train_losses_BN)/len(train_losses_BN)) 
  train_acc_epoch_BN.append(sum(train_acc_BN)/len(train_acc_BN))
  train_losses_epoch_GN.append(sum(train_losses_GN)/len(train_losses_GN)) 
  train_acc_epoch_GN.append(sum(train_acc_GN)/len(train_acc_GN))
  train_losses_epoch_LN.append(sum(train_losses_LN)/len(train_losses_LN)) 
  train_acc_epoch_LN.append(sum(train_acc_LN)/len(train_acc_LN)) 


# def test(model, device, test_loader):
#     model.eval()
#     test_loss = 0
#     correct = 0
#     with torch.no_grad():
#         for data, target in test_loader:
#             data, target = data.to(device), target.to(device)
#             output = model(data)
#             test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
#             pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
#             correct += pred.eq(target.view_as(pred)).sum().item()
#             # processed += len(data)
#             # test_acc.append(100*correct/processed)

#     test_loss /= len(test_loader.dataset)
#     test_losses.append(test_loss)

#     print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
#         test_loss, correct, len(test_loader.dataset),
#         100. * correct / len(test_loader.dataset)))
    
#     test_acc.append(100. * correct / len(test_loader.dataset))

In [13]:
# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


In [14]:
batchnet = BatchNet()
groupnet = GroupNet()
layernet = LayerNet()

In [15]:
nn_model = Model(batchnet, groupnet, layernet)

#### Batch Net implementation

In [16]:
model = nn_model().to(device)
#summary(nn_model, input_size=(1, 28, 28))

TypeError: ignored

In [ ]:
optimizer = optim.SGD(batch_model.parameters(), lr=0.01, momentum=0.9)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)
EPOCHS = 3
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(batch_model, device, train_loader, optimizer, epoch, l1 = True)
    scheduler.step()
    test(batch_model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

EPOCH: 0


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Loss=0.06407332420349121 Batch_id=468 Accuracy=88.86: 100%|██████████| 469/469 [00:20<00:00, 23.16it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0791, Accuracy: 9786/10000 (97.86%)

EPOCH: 1


Loss=0.04572343826293945 Batch_id=468 Accuracy=97.53: 100%|██████████| 469/469 [00:20<00:00, 22.49it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0505, Accuracy: 9846/10000 (98.46%)

EPOCH: 2


Loss=0.03412448987364769 Batch_id=468 Accuracy=98.09: 100%|██████████| 469/469 [00:21<00:00, 21.88it/s]



Test set: Average loss: 0.0452, Accuracy: 9870/10000 (98.70%)



In [ ]:
super_train_trac.append(train_acc_epoch)
super_train_trlo.append(train_losses_epoch)
super_train_teac.append(test_acc)
super_train_telo.append(test_losses)

RuntimeError: ignored

##### Layer model

In [ ]:
# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)


layer_model = LayerNet().to(device)
summary(layer_model, input_size=(1, 28, 28))

In [ ]:
optimizer = optim.SGD(layer_model.parameters(), lr=0.01, momentum=0.9)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

EPOCHS = 3
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(layer_model, device, train_loader, optimizer, epoch)
    scheduler.step()
    test(layer_model, device, test_loader)

In [ ]:
second_model_trac = copy.deepcopy(train_acc_epoch)
second_model_trlo = copy.deepcopy(train_losses_epoch)
second_model_teac = copy.deepcopy(test_acc)
second_model_telo  = copy.deepcopy(test_losses)


In [ ]:
plt.subplot(1,2,1)
plt.plot(first_model_trac)
plt.plot(second_model_trac)

plt.subplot(1,2,2)
plt.plot(first_model_telo)
plt.plot(second_model_telo)

##### Group normalization

In [ ]:
group_model = GroupNet().to(device)
summary(group_model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
         GroupNorm-3            [-1, 8, 26, 26]              16
            Conv2d-4           [-1, 12, 24, 24]             864
              ReLU-5           [-1, 12, 24, 24]               0
         GroupNorm-6           [-1, 12, 24, 24]              24
            Conv2d-7            [-1, 8, 24, 24]              96
         MaxPool2d-8            [-1, 8, 12, 12]               0
            Conv2d-9           [-1, 12, 10, 10]             864
             ReLU-10           [-1, 12, 10, 10]               0
        GroupNorm-11           [-1, 12, 10, 10]              24
           Conv2d-12             [-1, 16, 8, 8]           1,728
             ReLU-13             [-1, 16, 8, 8]               0
        GroupNorm-14             [-1, 1

In [ ]:
optimizer = optim.SGD(group_model.parameters(), lr=0.01, momentum=0.9)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(group_model, device, train_loader, optimizer, epoch)
    scheduler.step()
    test(group_model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
         GroupNorm-3            [-1, 8, 26, 26]              16
            Conv2d-4           [-1, 12, 24, 24]             864
              ReLU-5           [-1, 12, 24, 24]               0
         GroupNorm-6           [-1, 12, 24, 24]              24
            Conv2d-7            [-1, 8, 24, 24]              96
         MaxPool2d-8            [-1, 8, 12, 12]               0
            Conv2d-9           [-1, 12, 10, 10]             864
             ReLU-10           [-1, 12, 10, 10]               0
        GroupNorm-11           [-1, 12, 10, 10]              24
           Conv2d-12             [-1, 16, 8, 8]           1,728
             ReLU-13             [-1, 16, 8, 8]               0
        GroupNorm-14             [-1, 1

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Loss=0.12374579161405563 Batch_id=468 Accuracy=86.31: 100%|██████████| 469/469 [00:13<00:00, 34.00it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0893, Accuracy: 9750/10000 (97.50%)

EPOCH: 1


Loss=0.026314927265048027 Batch_id=468 Accuracy=97.06: 100%|██████████| 469/469 [00:13<00:00, 33.90it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0254, Accuracy: 9927/10000 (99.27%)

EPOCH: 13


Loss=0.05491850897669792 Batch_id=468 Accuracy=99.14: 100%|██████████| 469/469 [00:13<00:00, 33.90it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0255, Accuracy: 9926/10000 (99.26%)

EPOCH: 14


Loss=0.03589383140206337 Batch_id=468 Accuracy=99.09: 100%|██████████| 469/469 [00:14<00:00, 32.83it/s]



Test set: Average loss: 0.0254, Accuracy: 9928/10000 (99.28%)

